In [ ]:
from fastai import *
from fastai.vision import *
from dfs import *
from metrics import *
from efnet import EfficientNet
from losses import qappa_loss, kapjhapLoss, kapjhapMse
import warnings

warnings.filterwarnings('ignore')

In [ ]:
bs = 22

model = EfficientNet.from_pretrained('efficientnet-b5',num_classes=1)


tfms = ([crop_pad(),rotate(degrees=360.),*zoom_crop(scale=(1,1.4), do_rand=True),
         contrast(scale=(0.75,1.5)),flip_lr(p=0.5)],[crop_pad()])
         

                                                            
data = (ImageList.from_df(df=df1,path='./',cols='path') 
        .split_by_rand_pct(0.1,seed=1)
        .label_from_df(cols='diagnosis',label_cls=FloatList)
        .transform(tfms,padding_mode='border') 
        .databunch(bs=bs,num_workers=6) 
        .normalize(imagenet_stats)  
       )

In [ ]:
learn = Learner(data,
                model,
                loss_func=nn.MSELoss(),
                metrics = [accuracy_met,kappa_score],
                callback_fns = ShowGraph,
                model_dir="weights")